In [2]:
import torch
import torchvision as tv

In [6]:
# Download test data MNIST
train_data = tv.datasets.MNIST(
  root="./data",
  train=True,
  download=True
)

In [7]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [8]:
# Normalize data
train_data = train_data.data.float() / 255
train_data.data

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [ ]:
class ClassicLeNet5(torch.nn.Module):
  def __init__(self, w: int, h: int, num_classes: int):
    super(ClassicLeNet5, self).__init__()
    self.conv1 = torch.nn.Conv2d(
      in_channels=1,
      out_channels=6, # number of kernels
      kernel_size=5,
      stride=1,
      padding=0
    )
    self.conv2 = torch.nn.Conv2d(
      in_channels=6,
      out_channels=16,
      kernel_size=5,
      stride=1,
      padding=0
    )
    self.fc1 = torch.nn.Linear(in_features=16 * 5 * 5, out_features=120)
    self.fc2 = torch.nn.Linear(in_features=120, out_features=84)
    self.fc3 = torch.nn.Linear(in_features=84, out_features=num_classes)

  def forward(self, x):
    # x = data passing through the network
    print(x.shape)
    x = torch.nn.functional.tanh(self.conv1(x))
    x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
    x = torch.nn.functional.tanh(self.conv2(x))
    x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
    print(x.shape)
    x = x.view(-1, 16 * 5 * 5)
    x = torch.nn.functional.tanh(self.fc1(x))
    x = torch.nn.functional.tanh(self.fc2(x))
    x = torch.nn.functional.softmax(self.fc3(x), dim=1)

    return x

In [ ]:
# instantiate model
model = ClassicLeNet5(w=28, h=28, num_classes=10)

In [9]:
model(train_data.data[0].unsqueeze(0).unsqueeze(0))

torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 4, 4])


tensor([[0.1071, 0.1003, 0.0977, 0.1032, 0.0868, 0.0958, 0.0941, 0.0984, 0.1055,
         0.1111]], grad_fn=<SoftmaxBackward0>)

In [10]:
# bigger batch size = more memory
train_dl = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

In [ ]:
# Training params and loss function
learning_rate = 0.001
num_epochs = 5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train data
for epoch in range(num_epochs):
  model.train()
  for images, labels in train_dl:
    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
